# This notebook uses Papermill to batch run models

In [ ]:
%load_ext lab_black
import papermill as pm
import os
from multiprocessing import Pool
import pandas as pd
import altair as alt

## Batch configurations

In [ ]:
import random
seeds = [int(random.random() * 1e5) for x in range(10)]

batch_name = "O2P_noreg"

# Create batch cfgs
batch_cfgs = []
i = 0

for noise in [0., 4.]:
    for h in [25, 100]:
        i += 1
        code_name = batch_name + "_model_{:04d}".format(i)

        batch_cfg = dict(
            sn=i,
            in_notebook="OSP_master.ipynb",
            code_name=code_name,
            model_folder="models/" + code_name + "/",
            out_notebook="models/" + code_name + "/output.ipynb",
            params=dict(
                code_name=code_name,
                sample_name='hs04',
                sample_rng_seed=4321,
                tf_rng_seed=4444,
                use_semantic=False,
                input_dim=119,
                hidden_units=h,
                output_dim=250,
                cleanup_units=50,
                use_attractor=False,
                rnn_activation='sigmoid',
                regularizer_const=5e-6,
                p_noise=noise,
                tau=0.2,
                max_unit_time=4.,
                n_mil_sample=2.,
                batch_size=128,
                learning_rate=0.005,
                save_freq=5,
                bq_dataset=batch_name
            )
        )
        batch_cfgs.append(batch_cfg)


# Run
def run_batch(cfg):
    try:
        print("Running model {}".format(cfg['sn']))

        if not os.path.exists(cfg['model_folder']):
            os.mkdir(cfg['model_folder'])

        pm.execute_notebook(
            cfg['in_notebook'],
            cfg['out_notebook'],
            parameters=cfg['params'],
        )

    except:
        print("Error occur in {}".format(cfg['code_name']))

## Run batch

In [ ]:
# Run in parallel pool
with Pool(4) as pool:
    pool.map(run_batch, batch_cfgs)
    

In [ ]:
# Push results to BQ
from meta import model_cfg, connect_gbq
import pandas as pd
from tqdm import tqdm

# Make connection to bq
bq = connect_gbq()

for sn in tqdm(range(len(batch_cfgs))):

    model_folder = 'models/{0:s}_model_{1:04d}'.format(batch_name, sn + 1)
    strain_i_hist = pd.read_csv(model_folder + '/result_strain_item.csv')
    grain_i_hist = pd.read_csv(model_folder + '/result_grain_item.csv')

    bq.push_all(
        batch_name, batch_cfgs[sn]['params'], strain_i_hist, grain_i_hist
    )

#### Shutdown compute engine

In [ ]:
from time import sleep
sleep(30)
!sudo poweroff  

## Compile results

In [ ]:
import os, json
import pandas as pd
from meta import connect_gbq
from evaluate import vis
from tqdm import tqdm

conn = connect_gbq()
cfgs = conn.read_bq_cfg(batch_name)

# Read cfg files from BQ
print('===== Batch level hyperparams (columns that have >1 unique value) =====')
for i, x in enumerate(cfgs.columns):
    if not x in ['code_name', 'uuid']:
        if len(cfgs[x].unique()) > 1:
            print(
                'Column <{}> has these unique values: {}'.format(
                    x, cfgs[x].unique()
                )
            )

# Parse each run by batch_eval, which aggregate item level data to condition level
# and merge Grain and Strain into one single file (Using local files instead of BQ,
# may use BQ for way way more data... >5Gbs I guess)

models_path = []
for i in range(len(cfgs)):
    models_path.append('models/' + batch_name + '_model_{0:04d}'.format(i + 1))

batch_acc = pd.DataFrame()

for i in tqdm(range(len(cfgs))):

    model_path = 'models/' + batch_name + '_model_{0:04d}'.format(i + 1)

    this_eval = vis(
        model_path, 'result_strain_item.csv', 'result_grain_item.csv'
    )  # Eval lesion and grain
    this_eval.parse_cond_df()
    batch_acc = pd.concat([batch_acc, this_eval.cdf], ignore_index=True)

df = pd.merge(batch_acc, cfgs, 'left', 'code_name')
print('Done')

In [ ]:
# Save to h5 format
df.to_hdf('batch_eval/{}_cdf.h5'.format(batch_name), key='df', mode='w')

### Plotting

In [ ]:
df.columns

In [ ]:
df.to_csv('tmp_check.csv')

In [ ]:
df

In [ ]:
import pandas as pd
df = pd.read_hdf('batch_eval/{}_cdf.h5'.format(batch_name), 'df')

# Plotting results
import altair as alt
from evaluate import make_df_wnw

alt.data_transformers.enable("default")
alt.data_transformers.disable_max_rows()

# Selectors for interactions
sel_run = alt.selection(type="multi", on="click", fields=["code_name"])
sel_cond = alt.selection(
    type="multi", on="click", fields=["cond"], bind="legend"
)

# Heatmap for final epoch & timestep (Overview)
plot_timestep = df.timestep.max()
# plot_timestep = 19

# Plot strain

df_ov = df[(df.epoch == df.epoch.max()) & (df.timestep == plot_timestep)]

overview = (
    alt.Chart(df_ov).mark_rect().encode(
        x="hidden_units:O",
        y="p_noise:O",
        column='cond',
        row='exp',
        color=alt.Color("acc", scale=alt.Scale(scheme="redyellowgreen")),
        opacity=alt.condition(sel_run, alt.value(1), alt.value(0.1)),
        tooltip=["code_name", "acc"],
    ).add_selection(sel_run).properties(title="Overall Strain accuracy")
)

# df_ov_strain = df[(df.epoch == df.epoch.max()) &
#                   (df.timestep == plot_timestep) & (df.exp == 'strain')]

# overview_strain = (
#     alt.Chart(df_ov_strain).mark_rect().encode(
#         x="hidden_units:O",
#         y="p_noise:O",
#         column='cond',
#         color=alt.Color("acc", scale=alt.Scale(scheme="redyellowgreen")),
#         opacity=alt.condition(sel_run, alt.value(1), alt.value(0.1)),
#         tooltip=["code_name", "acc"],
#     ).add_selection(sel_run).properties(title="Overall Strain accuracy")
# )

# df_ov_grain = df[(df.epoch == df.epoch.max()) & (df.timestep == plot_timestep) &
#                  (df.exp == 'grain')]

# overview_grain = (
#     alt.Chart(df_ov_grain).mark_rect().encode(
#         x="hidden_units:O",
#         y="p_noise:O",
#         column='cond',
#         color=alt.Color("acc", scale=alt.Scale(scheme="redyellowgreen")),
#         opacity=alt.condition(sel_run, alt.value(1), alt.value(0.1)),
#         tooltip=["code_name", "acc"],
#     ).add_selection(sel_run).properties(title="Overall Grain accuracy")
# )

# Accuracy over epoch at last time step for selected model
last_time_point = df[df.timestep == df.timestep.max()]

acc_plot = (
    alt.Chart(last_time_point).mark_line(point=True).encode(
        y=alt.Y("acc:Q", scale=alt.Scale(domain=(0, 1))),
        x="epoch",
        color="cond",
        opacity=alt.condition(sel_cond, alt.value(1), alt.value(0.1)),
        tooltip=["code_name", "acc"],
    ).add_selection(sel_cond).transform_filter(sel_run).properties(
        title="Full model at final time step"
    )
)

# Accuracy Word (HF-INC) vs. Nonwords
wnw_df = make_df_wnw(df, selected_cond=['INC_HF', 'ambiguous', 'unambiguous'])

wnw_plot = (
    alt.Chart(wnw_df).mark_point().encode(
        y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
        x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
        color=alt.Color("epoch", scale=alt.Scale(scheme="redyellowgreen")),
        tooltip=["epoch", "word_acc", "nonword_acc"],
    ).transform_filter(sel_run).properties(
        title="Word vs. Nonword accuracy at final time step"
    )
)

# Plot diagonal
diagline = alt.Chart(pd.DataFrame({
    'x': [0, 1],
    'y': [0, 1]
})).mark_line().encode(x='x', y='y')

wnw_with_diag = wnw_plot + diagline

# overview = overview_strain & overview_grain
mainplots = acc_plot & wnw_with_diag
plot = overview | mainplots
plot

# plot.save('batch_eval/O2P_main_all_HFINC_NW.html')

In [ ]:
cov_dict = {
    'O2P_noreg_model_0001': 'Hidden 25, noise 0',
    'O2P_noreg_model_0002': 'Hidden 100, noise 0',
    'O2P_noreg_model_0003': 'Hidden 25, noise 4',
    'O2P_noreg_model_0004': 'Hidden 100, noise 4',
}

# Better label
wnw_df['group'] = wnw_df.code_name.map(lambda x: cov_dict[x])

In [ ]:
wnw_for_epoch_plot = wnw_df.melt(
    id_vars=['group', 'epoch'],
    value_vars=['word_acc', 'nonword_acc'],
    var_name='wnw',
    value_name='acc'
)

wnw_for_epoch_plot['group_wnw'
                  ] = wnw_for_epoch_plot.group + wnw_for_epoch_plot.wnw

sel_group = alt.selection(
    type="multi", on="click", fields=["group_wnw"], bind="legend"
)

plot_epoch = alt.Chart(wnw_for_epoch_plot).mark_line(point=True).encode(
    y=alt.Y("acc:Q", scale=alt.Scale(domain=(0, 1))),
    x="epoch:Q",
    color="group_wnw:N",
    shape="wnw",
    opacity=alt.condition(sel_group, alt.value(1), alt.value(0.1)),
    tooltip=["epoch", "acc"],
).add_selection(sel_group).properties(
    title="Plot word and nonword accuracy by epoch"
)

plot_epoch.save('batch_eval/combined4runs_plot_epoch.html')
plot_epoch

### Plot Word vs. Nonword across 4 runs

In [ ]:
# Plot
sel_group = alt.selection(
    type="multi", on="click", fields=["group"], bind="legend"
)

plot_wnw = alt.Chart(wnw_df).mark_line(point=True).encode(
    y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
    x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
    color="group:N",
    opacity=alt.condition(sel_group, alt.value(1), alt.value(0.1)),
    tooltip=["epoch", "word_acc", "nonword_acc"],
).add_selection(sel_group).properties(
    title="Word vs. Nonword accuracy at final time step"
)

plot_wnw.save('batch_eval/combined4runs_plot_wnw.html')

plot_wnw

## Eval one model

In [ ]:
from meta import model_cfg
from evaluate import vis

code_name = '{0:}_model_{1:04d}'.format(batch_name, 47)

# Load cfg from json
cfg = model_cfg(None)
cfg.load_cfg_json('models/' + code_name + '/model_config.json')

vis = vis(
    cfg.path_model_folder, 'result_strain_item.csv', 'result_grain_item.csv'
)

vis.parse_cond_df()

full = vis.plot_dev_interactive('acc').properties(title='Full input')

full

### Flexible development plot

In [ ]:
vis.plot_dev('acc', exp=None, condition='cond', timestep=2)

### Flexible time plot

In [ ]:
vis.plot_time('acc', exp='strain', condition='cond', epoch=400)